In [1]:
# For Google Colab use
import os 
try:
      from google.colab import drive
      drive.mount('/content/drive', force_remount=True)
      %cd "/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/"
except ModuleNotFoundError:
    pass

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp


In [2]:
!pip install face_recognition

In [3]:
# Imports
import re
from heapq import nsmallest
from math import ceil
from time import time
from random import randint, seed
from pipeline import *
import ntpath
from tqdm import tqdm
from PIL import Image
import gc
from psutil import virtual_memory

In [4]:
# Pick a number of frames (subsection) to create each image from
# `interval_in_frames = 1` = extract every frame
# `interval_in_frames = 60` = extract every 1 second chunk (30fps)
interval_in_frames = 30

In [5]:
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

def get_filename(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def get_parentdir_path(path):
    split_path = path.split(get_filename(p))[0]
    return split_path

def get_target_dir(path):
    fname = f"/{get_filename(path)}"
    rnd_dir = f"{path.replace(fname, '')}_rnd/"
    avg_dir = f"{path.replace(fname, '')}_avg/"
    return rnd_dir, avg_dir

def get_target_path(path):
    fname = get_filename(path).replace(".mp4", ".png")
    rnd_dir, avg_dir = get_target_dir(path)
    rnd_pth = f"{rnd_dir}{fname}"
    avg_pth = f"{avg_dir}{fname}"
    return rnd_pth, avg_pth

all_files_in_cwd = [f for f in absoluteFilePaths(".")]
mp4_paths = [f for f in all_files_in_cwd if "mp4" in f]
videos_to_do = sorted(mp4_paths)

# Tests
p = "/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences/619.mp4"
p, get_filename(p), get_parentdir_path(p), get_target_path(p), get_target_dir(p)


('/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences/619.mp4',
 '619.mp4',
 '/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences/',
 ('/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences_rnd/619.png',
  '/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences_avg/619.png'),
 ('/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences_rnd/',
  '/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences_avg/'))

In [6]:
videos_to_do_4tup = []
for v in videos_to_do:
    rdir, adir = get_target_dir(v)
    rpth, apth = get_target_path(v)
    videos_to_do_4tup += [(v, rdir, adir, rpth, apth)]

gc.collect()

61

In [7]:
def maintain_aspect_ratio_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    return cv2.resize(image, dim, interpolation=inter)

In [8]:
# video_dir_path, interval = '/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/FaceShifter/000_000.mp4', 1
# video_dir_path, interval = '/content/drive/MyDrive/Colab Notebooks/MInf2 Colab/Datasets/FaceForensicspp/original_sequences/01__walking_and_outside_surprised.mp4', 1
# frames = []
# vidcap = cv2.VideoCapture(video_dir_path)
# success, image = vidcap.read()
# while success:
#     # h, w, _ = image.shape
#     # if w*h > 465920:
#         # for H in list(range(1, h))[::-1]:
#             # if int(w*(H / float(h))) * H <= 1920*1080+99999:
#     print(image.shape)
#     image = maintain_aspect_ratio_resize(image, height=1080+50) 
#     print(image.shape)
#     frames.append(image[:, :, ::-1])
#     success, image = vidcap.read()
#     break


In [9]:
# for i, video_dir_path in tqdm(enumerate(videos_to_do)):
for i, (video_dir_path, rnd_dir, avg_dir, rnd_pth, avg_pth) in tqdm(enumerate(videos_to_do_4tup), total=len(videos_to_do_4tup)):
    # print("\n", video_dir_path, "\n", rnd_dir, "\n", avg_dir, "\n", rnd_pth, "\n", avg_pth, "\n")

    # Sanity checks
    for dir in (rnd_dir, avg_dir):
        if not os.path.exists(dir):
            os.makedirs(dir)

    # files_exists = os.path.isfile(rnd_pth.replace(".png", "_0.png")) and os.path.isfile(avg_pth.replace(".png", "_0.png"))
    # if files_exists:
        # print(f"\nskipping & deleting {video_dir_path}")
        # os.remove(video_dir_path)
        # os.remove(video_dir_path)
        # continue

    # Locate video and extract all frames
    # frames = get_every_frame(video_dir_path, interval=1)
    # size = 0
    frames = []
    vidcap = cv2.VideoCapture(video_dir_path)
    success, image = vidcap.read()
    while success:
        # h, w, _ = image.shape
        # if w*h > 465920:
        #     for H in list(range(1, h))[::-1]:
        #         if int(w*(H / float(h))) * H <= 465920:
        image = maintain_aspect_ratio_resize(image, height=512)
                    # break
        frames.append(image[:, :, ::-1])
        success, image = vidcap.read()


    # Use frames to find all face locations
    face_coordinates = get_face_locations(frames, GPU=True)
    # print(len(face_coordinates))

    save_faces(sub_average(frames, face_coordinates, interval=interval_in_frames), avg_pth.replace(".png", ""))
    save_faces(sub_frames_every_interval(frames, face_coordinates, interval=interval_in_frames), rnd_pth.replace(".png", ""))

    # Free memory
    if virtual_memory().percent >= 85:
        del frames
        del face_coordinates
        gc.collect()

    # delete video if both pngs exist
    # print(rnd_pth.replace(".png", ""))
    files_exists = os.path.isfile(rnd_pth.replace(".png", "_0.png")) and os.path.isfile(avg_pth.replace(".png", "_0.png"))
    # files_exists = os.path.isfile(rnd_pth.replace(".png", "_0.png")) or 
    if files_exists:
        # print(f"\ndeleting {video_dir_path}")
        os.remove(video_dir_path)
print("Done")

100%|██████████| 4/4 [01:31<00:00, 22.82s/it]

Done
